In [63]:
import pandas as pd
import numpy as np

In [64]:
### Подготовка данных на вход сети

protein_struct_loaded = pd.read_csv('protein_struct.csv')
#protein_struct_loaded = protein_struct_loaded.head(20000)
#protein_struct_loaded = protein_struct_loaded.head(500)

/kernel/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [65]:
#protein_struct_loaded = protein_struct_loaded.head(50)


In [66]:
len(protein_struct_loaded.protein_chain_id.unique())

25455

In [67]:
### y
data = protein_struct_loaded[['protein_chain_id','protein_chain_sequence','struct_class','struct_sequence','init_seq_num','term_seq_num']]
#data = data[(data.struct_class == '1') | (data.struct_class == '5') | (data.struct_class == 'sh')]

#data.struct_class.unique()
print(data.struct_class.groupby(data.struct_class).count())

struct_class
0         11
1     169932
3          6
4          7
5      59419
10         1
1       6179
5       1687
sh    242046
Name: struct_class, dtype: int64


In [68]:
data1= data.copy()

In [69]:
data1['ccc'] = data1.term_seq_num - data1.init_seq_num

In [70]:
print(data1[['struct_class','ccc']].ccc.groupby(data1.struct_class).agg(('count', 'sum')))

               count      sum
struct_class                 
0                 11       88
1             169932  2076368
3                  6       21
4                  7       74
5              59419   219777
10                 1        6
1               6179    76682
5               1687     6104
sh            242046  1081898


In [77]:
helix1 = 2076368+76682
helix5 = 219777+6104
sheet = 1081898
all = 25455*700
other = all - helix1 - helix5 - sheet


In [76]:
helix1,helix5,sheet, other

(2153050, 225881, 1081898, 14357671)

In [78]:
helix1/all,helix5/all,sheet/all, other/all

(0.12083228105620562,
 0.012676768527092628,
 0.060717681061817776,
 0.8057732693548839)

In [82]:
(all-helix1)/all,(all-helix5)/all,(all-sheet)/all, (all-other)/all

(0.8791677189437944,
 0.9873232314729073,
 0.9392823189381823,
 0.19422673064511603)

In [85]:
data['h1'] = '0'
data['h1_bin'] = '0'

data['h5'] = '0'
data['h5_bin'] = '0'

data['sh'] = '0'
data['sh_bin'] = '0'

#data['other_bin'] = '0'
max_protein_length = 700

<ipython-input-50-47b5bf3f6572>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['h1'] = '0'
<ipython-input-50-47b5bf3f6572>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['h1_bin'] = '0'
<ipython-input-50-47b5bf3f6572>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [86]:
data.shape[0]

479288

In [ ]:
#!c1.32
for i in range(0,data.shape[0]):
    data['sequence_length'] = len(data['protein_chain_sequence'].iloc[i])
    
    if ((data['struct_class'].iloc[i] == 1) | (data['struct_class'].iloc[i] == '1')):
        start_pos = data['init_seq_num'].iloc[i]-1
        stop_pos = data['term_seq_num'].iloc[i]
        data['h1'].iloc[i] = ''.ljust(min(start_pos,max_protein_length), '0').ljust(min(stop_pos,max_protein_length),'1').ljust(max_protein_length, '0')
        
    else:
        data['h1'].iloc[i] = ''.ljust(max_protein_length, '0')
        
    if  ((data['struct_class'].iloc[i] == 5) | (data['struct_class'].iloc[i] == '5')):
        start_pos = data['init_seq_num'].iloc[i]-1
        stop_pos = data['term_seq_num'].iloc[i]
        data['h5'].iloc[i] = ''.ljust(min(start_pos,max_protein_length), '0').ljust(min(stop_pos,max_protein_length), '1').ljust(max_protein_length, '0')
    else:
        data['h5'].iloc[i] = ''.ljust(max_protein_length, '0')
   
    if  data['struct_class'].iloc[i] == 'sh':
        start_pos = data['init_seq_num'].iloc[i]-1
        stop_pos = data['term_seq_num'].iloc[i]
        data['sh'].iloc[i] = ''.ljust(min(start_pos,max_protein_length), '0').ljust(min(stop_pos,max_protein_length), '1').ljust(max_protein_length, '0')
    else:
        data['sh'].iloc[i] = ''.ljust(max_protein_length, '0')
    
 
    data['h1_bin'].iloc[i] = np.array([int(x) for x in  data['h1'].iloc[i]])
    data['h5_bin'].iloc[i] = np.array([int(x) for x in  data['h5'].iloc[i]])
    data['sh_bin'].iloc[i] = np.array([int(x) for x in  data['sh'].iloc[i]])
    
data
#data.head(2)

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
#!c1.32

y_prep1 = data[['protein_chain_id','h1_bin','h5_bin','sh_bin']].groupby('protein_chain_id').sum()
y = y_prep1
y['y_labels'] = y['h1_bin']
y['other'] = y['h1_bin']
for i in range(0,y.shape[0]):
    #print(i)
    y['other'].iloc[i] =  y['h1_bin'].iloc[i] + y['h5_bin'].iloc[i] + y['sh_bin'].iloc[i]
    y['other'].iloc[i] = [abs(x-1) for x in y['other'].iloc[i]]
    y['y_labels'].iloc[i] = np.vstack((y['h1_bin'].iloc[i],y['h5_bin'].iloc[i],y['sh_bin'].iloc[i],y['other'].iloc[i])).T    
y = y.drop(columns = ['h1_bin','h5_bin','sh_bin','other'])
#y.head()


In [ ]:
### X
X_prep1 = protein_struct_loaded[['protein_chain_id','protein_chain_sequence']].groupby('protein_chain_id').agg('max')
X_prep1['protein_chain_seq'] = X_prep1.protein_chain_sequence[:].str[:max_protein_length]
X_prep1 = X_prep1.drop(columns = ['protein_chain_sequence'])
#X_prep1.head(6)

In [ ]:
simbs = np.sum(X_prep1.protein_chain_seq)
simbs_arr = np.array([x for x in  simbs])
simbs_unique = np.unique(simbs_arr)
simbs_unique = np.append('_',simbs_unique)
simbs_unique = pd.Series(simbs_unique)
#simbs_unique

In [ ]:
len(simbs_unique)

In [ ]:
X_prep2 = X_prep1.copy()
X_prep2['chain_simbs'] = [np.empty(0,dtype=float)]*len(X_prep2) 
X_prep2['chain_simbs_indexed'] =  [np.empty(0,dtype=float)]*len(X_prep2) 

#добавляю столбцы A,A_chain_freq
for s in simbs_unique:  
    #print(s)
    s_column = s + '_column'
    s_count_in_protein = s + '_count_in_protein'
    s_pssm = s + '_pssm'
    
    X_prep2.insert(1, s_column,  [np.empty(0,dtype=float)]*len(X_prep2))  
    X_prep2.insert(1, s_count_in_protein,  0)
    X_prep2.insert(1, s_pssm,  [np.empty(0,dtype=float)]*len(X_prep2))

#print(X_prep2.columns)

In [ ]:
#!c1.32
simb_to_index = lambda x: simbs_unique[simbs_unique == x].index[0]
simb_to_1 = lambda x : 1 if (x == s) else 0

for i in range(0,X_prep2.shape[0]):
   #print(i)
    X_prep2['chain_simbs'].iloc[i] = np.array([x for x in  X_prep2.protein_chain_seq.iloc[i]]) 
    #print(X_prep2['chain_simbs'].iloc[i] )
    if len(X_prep2['chain_simbs'].iloc[i] ) <  max_protein_length:
        #dops = ['_']*(max_protein_length - len(arr_chain))
        X_prep2['chain_simbs'].iloc[i] = np.append(X_prep2['chain_simbs'].iloc[i], 
                                                   ['_']*(max_protein_length - len(X_prep2['chain_simbs'].iloc[i])))

    X_prep2['chain_simbs_indexed'].iloc[i] =np.array([simb_to_index(x) for x in  X_prep2['chain_simbs'].iloc[i]])

    for s in simbs_unique: 
        s_column = s + '_column'
        s_count_in_protein = s + '_count_in_protein'
        X_prep2[s_column].iloc[i] = np.array([simb_to_1(x) for x in  X_prep2['chain_simbs'].iloc[i]])
        X_prep2[s_count_in_protein].iloc[i] = sum(X_prep2[s_column].iloc[i])
    
#X_prep2.head(3)

In [ ]:
#X = X_prep2.drop(columns=['chain_simbs','chain_array_for_pssm'])
X = X_prep2[['protein_chain_seq','chain_simbs_indexed']]
#X.head(5)#.loc['1zg2_A'].chain_simbs_indexed

In [ ]:
X.chain_simbs_indexed[0].shape

In [ ]:
# аггрегирую 
SPFM = X_prep2.drop(columns =['protein_chain_seq','chain_simbs','chain_simbs_indexed']).sum()
#SPFM.head(2)    

In [ ]:
all_simb_count = max_protein_length * X_prep2.shape[0]
norm_to_log2 = lambda x : np.log2(x)
pssm_column_array = list()
print(pssm_column_array, type(pssm_column_array))
for s in simbs_unique: 
    #print(s)
    s_column = s + '_column'
    s_count_in_protein = s + '_count_in_protein'
    s_column_freq = s + '_column_freq'    # доля присутствия символа на каждой позиции в датасете
    s_overall_frequences = s + '_overall_frequences'  #доля присутствия символа вообще в датасете
    s_normalized = s + '_normalized'
    s_log2 = s + '_log2'
    column_pssm = s + '_pssm'  #для создания списка столбцов, которые потом надо будет сложить в X_prep2
    
    SPFM[s_column_freq] = SPFM[s_column] / X_prep2.shape[0]
    SPFM[s_overall_frequences] = SPFM[s_count_in_protein] / all_simb_count
    SPFM[s_normalized] = SPFM[s_column_freq] / SPFM[s_overall_frequences] + 0.01
    #print(SPFM[[s_column,s_normalized]])
    SPFM[s_log2]= np.log2(SPFM[s_normalized])   #([norm_to_log2(x) for x in  SPFM[s_normalized]])
    #print(SPFM[s_log2])
    
    pssm_column_array.append(column_pssm)

#SPFM


In [ ]:
SPFM = pd.DataFrame(SPFM).T
#print(SPFM)

In [ ]:
X_prep2['chain_pssm'] = X_prep2['chain_simbs_indexed']
X_prep2['chain_pssm_sum'] = [np.empty(0,dtype=float)]*len(X_prep2) 

In [ ]:
#!c1.32
# получаю PSSM 
#simb_to_pos = lambda spos : SPFM[s+'_log2'] if (spos == s) 
#SPFM = pd.DataFrame(SPFM).T
#X_prep2['chain_pssm'] = X_prep2['chain_simbs_indexed']
#X_prep2['chain_pssm_sum'] = [np.empty(0,dtype=float)]*len(X_prep2) 
for i in range(0,X_prep2.shape[0]):
    #(i, X_prep2.chain_simbs.iloc[i])
    #print(X_prep2.chain_simbs.iloc[i].shape[0])
    #X_prep2['chain_pssm'].iloc[i] = None
    for s in simbs_unique: 
        s_pssm = s + '_pssm'
        s_column = s + '_column'
        s_log2 = s + '_log2'
        X_prep2[s_pssm].iloc[i] = np.array(X_prep2[s_column].iloc[i] * SPFM[s_log2][0])
        X_prep2['chain_pssm'].iloc[i] = np.vstack((X_prep2['chain_pssm'].iloc[i],X_prep2[s_pssm].iloc[i]))
      #  X_prep2['chain_pssm_sum'].iloc[i] = X_prep2[['chain_pssm_sum',s_pssm]].iloc[i].sum(axis=1)
        
    X_prep2['chain_pssm'].iloc[i] = X_prep2['chain_pssm'].iloc[i][1:]
    X_prep2['chain_pssm_sum'].iloc[i] = X_prep2.chain_pssm.iloc[i].sum(axis=0)


In [ ]:
#!c1.32
PSSM = X_prep2[['protein_chain_seq','chain_pssm_sum']]

## Построение нейронной сети для предсказания вторичных структур по последовательности аминокислотных остатков

In [ ]:
dataX = X
datay = y
dataPSSM = PSSM

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

In [ ]:
class ChainDataset(Dataset):
    def __init__(self, transform=None):
        self.Xs = dataX
        self.ys = datay        
        self.PSSM = dataPSSM
        self.transform = transform
        
    def __len__(self):
        return len(self.Xs)
        
    def __getitem__(self,idx):
        chain_id = self.Xs.index[idx]
        chain_sequence = self.Xs.protein_chain_seq[idx]
        X = self.Xs.chain_simbs_indexed[idx]   #700x1        
        y = self.ys.y_labels[idx] #700x2
        PSSM = self.PSSM.chain_pssm_sum[idx]      
        # print(y)
        sample = {'chain_id': chain_id, 'chain_sequence': chain_sequence ,'inputs': X, 'labels': y, 'pssm': PSSM}
        return(sample)

In [ ]:
data_transform = transforms.Compose([
        transforms.ToTensor()
    ])

In [ ]:
dataset = ChainDataset(transform = data_transform)
dataset.__getitem__(0)['inputs'].shape 

In [ ]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, 
                                                    [train_size, test_size], 
                                                    generator=torch.Generator().manual_seed(42))

In [ ]:
batch_size = 50

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
from torch.nn import Sequential

In [ ]:
class  ChainModel(nn.Module):
    def __init__(self):         
        super().__init__()
               
        self.x_embedding = nn.Embedding(24, 47)
        self.pssm_embedding = nn.Embedding(24, 24)
        
        self.conv3 = nn.Sequential(nn.Conv1d(in_channels=71, out_channels=68, kernel_size=3, padding=1),
                                   nn.ReLU())
        self.conv7 = nn.Sequential(nn.Conv1d(in_channels=71, out_channels=64, kernel_size=7, padding=3),
                                   nn.ReLU())
        self.conv11 = nn.Sequential(nn.Conv1d(in_channels=71, out_channels=60, kernel_size=11, padding=5),
                                   nn.ReLU())
        
        self.iL1= 192 # self.input_size_encoder
        self.oL1= 192 # self.hidden_size
        self.iL2= self.iL1*2 + self.oL1*2 #(Bidirectional)
        self.oL2= self.iL2 
        self.iL3= self.oL2*2 + self.oL1*2 + self.iL1*2   #(Bidirectional)
        self.oL3= self.iL3

        self.gru1 = nn.Sequential(nn.Dropout(p=0.2, inplace=False), nn.GRU(input_size=self.iL1, hidden_size=self.oL1, batch_first=True, bidirectional=True))#, nn.Dropout(p=0.2, inplace=False) ) 
        self.gru2 = nn.Sequential(nn.Dropout(p=0.2, inplace=False), nn.GRU(input_size=self.iL2, hidden_size=self.oL2, batch_first=True, bidirectional=True))#, nn.Dropout(p=0.2, inplace=False) )  
        self.gru3 = nn.Sequential(nn.Dropout(p=0.2, inplace=False), nn.GRU(input_size=self.iL3, hidden_size=self.oL3, batch_first=True, bidirectional=True))#, nn.Dropout(p=0.2, inplace=False) ) 
    
        
        #input_size – The number of expected features in the input x
        #hidden_size – The number of features in the hidden state h
        #num_layers – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two GRUs together to form a stacked GRU, with the second GRU taking in outputs of the first GRU and computing the final results. Default: 1
        #bias – If False, then the layer does not use bias weights b_ih and b_hh. Default: True
        #batch_first – If True, then the input and output tensors are provided as (batch, seq, feature) instead of (seq, batch, feature). Note that this does not apply to hidden or cell states. See the Inputs/Outputs sections below for details. Default: False
        #dropout – If non-zero, introduces a Dropout layer on the outputs of each GRU layer except the last layer, with dropout probability equal to dropout. Default: 0
        #bidirectional – If True, becomes a bidirectional GRU. Default: False
        
        self.detection = nn.Sequential( nn.Linear(4608, 128),   #700x192 =      
                                        nn.BatchNorm1d(max_protein_length),
                                        nn.Linear(128, 4),
                                        nn.BatchNorm1d(max_protein_length),  
                                        nn.Softmax(dim = 2)                         
                                    )
                                 
                                 
    def forward(self, x, PSSM):
        
             
        x_emb = self.x_embedding(x) 
      
        PSSM_emb = self.pssm_embedding(x) 
      
        
        x_emb = torch.cat((x_emb.float(), PSSM_emb.float()),dim=2)#  print("x_emb",x_emb)
        ####print(x_emb.shape)
        #print(x_emb)
        x_emb = x_emb.permute(0,2,1)
        #print(x_emb.shape)
        #print(x_emb)
        xc3 = self.conv3(x_emb)  # barch_size X 700 X 71 -> barch_size X 700 X 68
        xc7 = self.conv7(x_emb)   
        xc11 = self.conv11(x_emb)  
        
        x_conv = torch.cat((xc3, xc7, xc11),1)
        x_conv = x_conv.permute(0,2,1)

        output1, h_n1 = self.gru1(x_conv) 
        fw1_res = torch.cat((x_conv,output1[:,:,self.oL1:]), dim=2) # x,L1
        bw1_res = torch.cat((output1[:,:,:self.oL1],x_conv) ,dim=2) # L1,x
        output1_residual = torch.cat((fw1_res,bw1_res),dim=2)
        output2, h_n2 = self.gru2(output1_residual)
        
        fw2_res = torch.cat((x_conv,output1[:,:,self.oL1:],output2[:,:,self.oL2:]),dim=2) # x,L1,L2
 
        bw2_res = torch.cat((output2[:,:,:self.oL2],output1[:,:,:self.oL1],x_conv),dim=2) # L2,L1,x
        output2_residual = torch.cat((fw2_res,bw2_res),dim=2)
        output3, h_n3 = self.gru3(output2_residual) 
    
        x_detect = self.detection(output3)
        return x_detect

In [ ]:
import time
from sklearn.metrics import classification_report


In [ ]:
def train_nn(trainloader, model, opt, loss_fn):
    stime = time.monotonic()
    loss_train = []
    accuracy_train = []  
    accuracy_train_by_class = []
   
    model.train()
      
    for i, data in enumerate(trainloader, 0):
        
        inputs = data.get('inputs')
        labels = data.get('labels')           
        pssm = data.get('pssm')    
        
        predictions = model(inputs, pssm) 

        with torch.no_grad():
            predictions_max = torch.argmax(predictions,dim =2)
            labels_max = torch.argmax(labels,dim =2)     
           
            accuracy = accuracy_weighted(predictions.permute(0,2,1), labels_max)
            accuracy_by_class = accuracy_none(predictions.permute(0,2,1), labels_max)
                        
        
        loss =  loss_fn(predictions.permute(0,2,1), labels_max).mean()
        
      
        print(i, "TRAIN   LOSS", loss, " acc_weighted", accuracy,  " acc_by_class", accuracy_by_class) #, "loss_none", loss_none)
        opt.zero_grad()        
        loss.mean().backward()   
        opt.step()
    
        loss_train.append(loss.item())
        accuracy_train.append(accuracy.item())        
        accuracy_train_by_class.append(accuracy_by_class) 
        

    return model, loss_train, accuracy_train, accuracy_train_by_class 


def eval_nn(testloader, model, loss_fn):
    loss_test = []
    accuracy_test = []    
    accuracy_test_by_class = []
 
    model.eval()
    
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):

            inputs = data.get('inputs')
            labels = data.get('labels')
            pssm = data.get('pssm')

            predictions = model(inputs,pssm) 
            predictions_max = torch.argmax(predictions,dim =2)

            labels_max = torch.argmax(labels,dim =2)
         
            loss =  loss_fn(predictions.permute(0,2,1), labels_max).mean()
            accuracy = accuracy_weighted(predictions.permute(0,2,1), labels_max)
            accuracy_by_class = accuracy_none(predictions.permute(0,2,1), labels_max)              
            
            print( i ,"TEST LOSS", loss, " acc_weighted", accuracy,  " acc_by_class", accuracy_by_class)
          
            loss_test.append(loss.item())
            accuracy_test.append(accuracy.item())
            accuracy_test_by_class.append(accuracy_by_class)         
            
    return loss_test, accuracy_test, accuracy_test_by_class 

In [ ]:
def training_loop(model, trainloader, testloader, opt, loss_fn, max_epochs=2):
   
    loss_epoch_train = []
    loss_epoch_test = []
    acc_epoch_train = []
    acc_epoch_test = []
    acc_epoch_train_by_class = []
    acc_epoch_test_by_class = []
    
    for epoch in range(max_epochs):        
        model, loss_train, accuracy_train, accuracy_train_by_class = train_nn(trainloader, model, opt, loss_fn)
        loss_test, accuracy_test, accuracy_test_by_class  = eval_nn(testloader, model, loss_fn)

     
        loss_epoch_train.append(np.mean(loss_train))
        loss_epoch_test.append(np.mean(loss_test))

        acc_epoch_train.append(np.mean(accuracy_train))
        acc_epoch_test.append(np.mean(accuracy_test))

        acc_epoch_train_by_class.append(torch.mean(torch.reshape(torch.cat(accuracy_train_by_class,0),(-1,4)), axis = 0))
        acc_epoch_test_by_class.append(torch.mean(torch.reshape(torch.cat(accuracy_test_by_class,0),(-1,4)), axis = 0))
        
        print(f'Epoch: {epoch}\t'
                  f'Train loss: {loss_epoch_train[-1]:.5f}\t'
                  f'Test loss: {loss_epoch_test[-1]:.5f}\t'
                  f'Train accuracy: { acc_epoch_train[-1]:.2f}\t'
                  f'Test accuracy: { acc_epoch_test[-1]:.2f} ') #\t'
                
    return loss_epoch_train, loss_epoch_test, acc_epoch_train, acc_epoch_test, acc_epoch_train_by_class, acc_epoch_test_by_class

In [ ]:
model_chain = ChainModel()
weights = torch.tensor([0.88,  0.99,  0.94,  0.19])

loss_function = torch.nn.CrossEntropyLoss(weight=weights,reduction='none')#size_average=False,reduction='sum')#reduction='none')
optimizer = torch.optim.Adam(model_chain.parameters(), lr=0.0003) # lr=0.00005

import torchmetrics 
from torchmetrics import Accuracy
accuracy_none = Accuracy(average = 'none', num_classes = 4,  mdmc_average = 'samplewise')
accuracy_weighted = Accuracy(average = 'weighted', num_classes = 4,  mdmc_average = 'samplewise')


In [ ]:
loss_epoch_train, loss_epoch_test, acc_epoch_train, acc_epoch_test, acc_epoch_train_by_class, acc_epoch_test_by_class = training_loop(model_chain, train_loader, test_loader, optimizer, loss_function, max_epochs=6)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
def plot(loss_epoch_train, loss_epoch_test, acc_epoch_train, acc_epoch_test, acc_epoch_train_by_class, acc_epoch_test_by_class ):
    fig, ax = plt.subplots(2, 2, figsize=(20, 12))

    ax[0,0].plot(loss_epoch_train, label='Train loss')
    ax[0,0].plot(loss_epoch_test, label='Test loss')
    ax[0,0].legend()
    ax[0,0].grid()   
    ax[0, 0].set_title('Loss by epochs')

    ax[0,1].plot(acc_epoch_train, label='Train accuracy')
    ax[0,1].plot(acc_epoch_test, label='Test accuracy')
    ax[0,1].legend()
    ax[0,1].grid()
    ax[0, 1].set_title('Accuracy by epochs')
    
    ax[1][0].plot(acc_epoch_train_by_class[0], label='Train class0')    
    ax[1][0].plot(acc_epoch_train_by_class[1], label='Train class1')    
    ax[1][0].plot(acc_epoch_train_by_class[2], label='Train class2')    
    ax[1][0].plot(acc_epoch_train_by_class[3], label='Train class3')    
    ax[1][0].legend()
    ax[1][0].grid()   
    ax[1, 0].set_title('Accuracy by classes. Train')


    ax[1][1].plot(acc_epoch_test_by_class[0], label='Test class0')
    ax[1][1].plot(acc_epoch_test_by_class[1], label='Test class1')
    ax[1][1].plot(acc_epoch_test_by_class[2], label='Test class2')
    ax[1][1].plot(acc_epoch_test_by_class[3], label='Test class3')
    ax[1][1].legend()
    ax[1,1].grid()
    ax[1, 1].set_title('Accuracy by classes. Test')
    

In [ ]:

plot(loss_epoch_train, loss_epoch_test, acc_epoch_train, acc_epoch_test, acc_epoch_train_by_class, acc_epoch_test_by_class)